In [1]:
import numpy as np
import random
from scipy.special import gamma
import matplotlib.pyplot as plt
import healpy as hp


In [266]:
dLogEii = 0.1
LEd2 = 20;	LEd1 = 18.5
Nbins_det = (int) ( ( LEd2 - LEd1 ) / dLogEii + 0.5 )
r_istrpy = 120
density = 1.423e-07
Nsims = 1
dist_flag = 1
catalogue_flag = 1
fracs = np.array( ( 0.000000000648, 0.00000000000, 0.602136799733,\
                    0.32599207417, 0.070263992201 ) )
fracs_L = np.array( (  0.241086962941, 0.082710923419, 0.403818920296,\
                     0.272383193342, 0.000000000003 ))
#fracs = np.array( ( 1, 0., 0, 0, 0 ) )
#plt.rcParams['text.usetex'] = True

Nside = 32   #  for Healpix
Npix=12*Nside**2
print(Npix)

12288


In [267]:
class Flux_data:
    """
    Struct data to hold the values of the flux to be read,
    its energy, distance, weight of the flux
    """
    def __init__(self):
        """
        Initializes the object to 0 in all flieds
        Returns
        -------
        None.
        """
        self.Edet = 0
        self.dist = 0
        self.weight = 0
        self.magnitude = 0

In [268]:
class Source:
    """
    Object that will hold each source data.
    """
    def __init__(self, radius=0, RA=0, sinD=0):
        """
        Initializes an object of type source. 
        Parameters
        ----------
        radius : float, optional
            Distance to the source. The default is 0.
        RA : float, optional
            Right ascention. The default is 0.
        sinD : float, optional
            sin of the declination. The default is 0.
        Returns
        -------
        None.

        """
        self.sin_declination = sinD
        self.right_ascention = RA
        self.radius = radius
        self.weights = np.zeros((Nbins_det,3))
    
    def __add__(self, d1):
        """
        Overloads the + operator.
        Parameters
        ----------
        d1 : Source object to add to self.
        Returns
        -------
        Source
            Returns a Source object adding d1 weights.
        """
        for i,w in enumerate(d1.weights):
            self.weights[i][0]+= w[0]
            self.weights[i][1]+= w[1]
            self.weights[i][2]+= w[2]
        return self

In [269]:
def Source_distance( ds, r_iso ):
    """
    Creates an np.array of float of the distances to the sources.
    Parameters
    ----------
    ds : float
        Characteristic distance between the sources.
    r_iso : float
        Radius at which the flux becomes isotropic.
    Returns
    -------
    np.array of float
        Distances to the sources.
    """
    print("Simulation with random sources.\n")
    r_i = -1.;	n = 0;	i = 1
    R_i = np.empty(0)
    if dist_flag == 0 :
        aGamma = gamma(1./3.); bGamma = gamma(1)
        quotient = 0
        while ( r_i <= r_iso):
            quotient = aGamma/bGamma
            while (n <i):
                quotient = (3.*(i-n)-2)*quotient/(3.*(i-n))
                n=n+1
            quotient = quotient*i
            r_i = pow(3./(4.*np.pi), 1./3.)*ds*quotient
            if (r_i < r_istrpy):
                R_i = np.append(R_i, r_i)
            i+=1; n = 0
    elif dist_flag == 1 :
        Num_sources = int( 4 * np.pi * r_istrpy**3 * density / 3. + 0.5 )
        R_i = np.empty(0)
        R_i = r_istrpy * np.random.random( Num_sources )**(1./3)
    else:
        print("Incorrect distance flag")
    print( R_i.shape )
    return R_i



def Source_direction( R_i, sin_alpha = 1):
    """
    Creates a list of objects of type Source.
    Parameters
    ----------
    R_i : np.array of float of distances to the sources.
    Returns
    -------
    source_list : list of objects of type Source.
    """
    source_list = []
    for r_i in R_i:
        source = Source(0)
        source.right_ascention = 2. * np.pi* np.random.random()
        source.sin_declination = (2. * np.random.random() - 1.) * sin_alpha
        source.radius = r_i
        source_list.append( source )
    return source_list


def Source_from_catalogue():
    source_list = []
    print("Reading sources from catalogue.\n")
    input_name = "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/"+\
        "2MHL" + str( r_istrpy ) + "-236.txt"
    Num_sources = int( 4 * np.pi * r_istrpy**3 * density / 3. + 0.5 )
    alpha = 5.*np.pi/180
    Num_sources_gal = int( 4 * np.pi * r_istrpy**3 * np.sin(alpha) * density / 3. + 0.5 )
    Num_cat = Num_sources - Num_sources_gal
    try:
        with open( input_name ,'r') as file:
            for row in file:
                row = row.split('\t')
                source = Source(0)
                source.right_ascention = float( row[0] ) *np.pi/180
                source.sin_declination = np.sin( float(row[1])*np.pi/180 )
                source.radius = float( row[2] )
                source.magnitude = float( row[-1] )
                if source.radius < r_istrpy:
                    source_list.append( source )

            #source_list.sort( key=lambda x: x.magnitude, reverse = True )
            np.random.shuffle(source_list)
            print("Catalogue: ", len(source_list[ 0: Num_cat ]), ". Galactic: " , Num_sources_gal )
            source_list = source_list[ 0: Num_cat ].copy()
            R_i = r_istrpy * np.random.random(Num_sources_gal)**(1./3)
            source_list.extend( Source_direction(R_i, np.sin(alpha) ) )
            #for s in source_list:
            #	print(s.radius)
    except FileNotFoundError:
        print("File not found")
        return None
    except Exception as e:
        print(f"error as {e}")
        return None
    return source_list

In [270]:
def Load_weights(HE):
    print(HE)
    weights = np.zeros( ( 5, 5, Nbins_det, int(r_istrpy +1)) ) 
    cosines = np.zeros( ( 5, 5, Nbins_det, int(r_istrpy +1)) ) 
    flux = Flux_data()
    print("Loading weights :")
    for mass_index in range(0,5):
        if HE:
            input_name = "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1"+\
            "/MX_A" + str(massindex[mass_index][1]) + "H.txt"
            cosine_name = "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1"+\
            "/cosine_A" + str(massindex[mass_index][1]) + "H.txt"
        else:
            input_name = "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/"+\
            "/MX_A" + str(massindex[mass_index][1]) + "L.txt"
            cosine_name = "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1"+\
            "/cosine_A" + str(massindex[mass_index][1]) + "L.txt" 
        #print(input_name, end="\n\n")
        try:
            with open( input_name ,'r') as weight_file,\
                open( cosine_name ,'r') as cosine_file:
                for row_weight, row_cosine  in zip(weight_file, cosine_file):
                    row_weight = row_weight.split('\t')
                    row_cosine = row_cosine.split('\t')
                    flux.Edet = float(row_weight[0]); flux.dist = int(row_weight[1])
                    det_bin =  (int) (( flux.Edet - LEd1 ) / dLogEii)
                    for secondary in range(0, mass_index +1):
                        flux.weight = float( row_weight[secondary+2] )
                        cosine = float( row_cosine[secondary+2] )
                        if (flux.dist <= r_istrpy):
                            weights[ mass_index, secondary, det_bin, int(flux.dist) ] = flux.weight
                            cosines[ mass_index, secondary, det_bin, int(flux.dist) ] = cosine 
    
        except FileNotFoundError:
            print("File not found")
            print(input_name, cosine_name)
            return None
        except Exception as e:
            print(f"error as {e}")
            return None
    return weights, cosines

In [271]:
def Flux_ratio_calc(HE):
    """
    Calculates the total anisotropic flux and the total flux.
    Parameters
    ----------
    input_name : string
        Name of the file from which data will be read.
    mass_index : int
        Identifier of the mass group.
    Returns
    -------
    Flux_tot : np.array of float
        total flux from all sources at all distances.
    Flux_anistrpy : np.array of float
        total flux from all sources at distances such as R_source< R_isotropy
    """
    total_weight = np.zeros( (5, 5, Nbins_det) ) 
    istrpy_weight = np.zeros( (5, 5, Nbins_det) )
    total_flux = np.zeros( Nbins_det ); isotropic_flux = np.zeros( (5, Nbins_det) ) 
    
    print( "Calculating Flux ratio." )
    flux = Flux_data()
    for mass_index in range(0,5):
        if HE:
            input_name =  "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1"+\
            "/MX_A" + str(massindex[mass_index][1]) + "H.txt" 
        else:        
            input_name =  "/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1"+\
            "/MX_A" + str(massindex[mass_index][1]) + "L.txt" 
        print(input_name, end="\n\n")
        try:
            with open( input_name ,'r') as file:
                for row in file:
                    row = row.split('\t')
                    flux.Edet = float(row[0]); flux.dist = int(row[1])
                    det_bin =  (int) (( flux.Edet - LEd1 ) / dLogEii)
                    for secondary in range(0, mass_index +1):
                        flux.weight = float( row[secondary+2] )
                        total_weight[ mass_index, secondary, det_bin ] += flux.weight
                        if (flux.dist >= r_istrpy) :
                            #print(flux.dist)
                            istrpy_weight[ mass_index, secondary, det_bin ] += flux.weight
        except FileNotFoundError:
            print("File not found")
            return None
        except Exception as e:
            print(f"error as {e}")
            return None
    for mass_index in range(5):
        for secondary in range(mass_index+1):
            for binE in range(Nbins_det):
                isotropic_flux[secondary, binE] += fracs[mass_index]*\
                                       istrpy_weight[mass_index,secondary,binE]
    for mass_index in range(5):
        for secondary in range(mass_index+1):
            total_flux += fracs[mass_index]*total_weight[mass_index,secondary]
    return isotropic_flux, total_flux, istrpy_weight, total_weight

In [272]:
massindex = []
massindex.append( [ 1, 1 ] )
massindex.append( [ 2, 4 ] )
massindex.append( [ 7, 14 ] )
massindex.append( [ 14, 28] )	
massindex.append( [ 26, 56 ] )

weights_H, cosines_H = Load_weights(True)
weights_L, _ = Load_weights(False)

flux_H = np.zeros((Nbins_det)); flux_L = np.zeros((Nbins_det)); whtot = np.zeros((5,5,Nbins_det))
Q_H , Q_L =  5.55415e-38 , 1.44029e-36
for mass_index in range(5):
    for secondary in range(mass_index+1):
        for binE in range(Nbins_det):
            for dist in range(r_istrpy+1):
                whtot[mass_index, secondary, binE] += weights_H[mass_index, secondary, binE, dist]
                flux_H[binE] += fracs[mass_index] *\
                        weights_H[mass_index, secondary, binE, dist]
                flux_L[binE] += (Q_L/Q_H) * fracs_L[mass_index] *\
                        weights_L[mass_index, secondary, binE, dist]

N_sims = np.zeros (Nbins_det)
mean_dipole = np.zeros( ( 5, 5, Nbins_det) )
total_dipole = np.zeros(Nbins_det)
dipole_sims = np.zeros( (Nsims, Nbins_det) )
np.random.seed(1)

if catalogue_flag == 0:
	R_i = Source_distance( pow(density,-1./3), r_istrpy )
	sources = Source_direction( R_i )
else:
	sources = Source_from_catalogue()

#sources = sorted(sources, key=lambda x: x.radius, reverse=False)
#sources = sources[0:1]

for a,s in enumerate(sources):
	print(a+1, s.radius)


True
Loading weights :
False
Loading weights :
Reading sources from catalogue.

Catalogue:  1 . Galactic:  0
1 103.8


In [273]:
def calculate_flux(weights_H, sources, isotropy_weight_H, total_weight_H, flux_H, flux_L):
    halo_flux_source = np.zeros(( 5, Nbins_det, len(sources) ))
    flux_per_source = np.zeros(( 5, 5, Nbins_det, len(sources) ))
    model_flux = np.zeros(( 5, 5, Nbins_det ))
    for s,source in enumerate(sources):
        for primary in range(5):
            for secondary in range(primary+1):
                for binE in range(Nbins_det):
                    flux_per_source[primary,secondary,binE, s] =\
                        weights_H[primary,secondary, binE, int(source.radius) ]/source.radius**2
                    model_flux[primary,secondary,binE] +=\
                        flux_per_source[primary,secondary,binE, s]
    for s,source in enumerate(sources):
        for mass_index in range(5):
            for secondary in range(mass_index+1):
                for binE in range(Nbins_det):
                    if model_flux[mass_index, secondary, binE] != 0:
                        J_r = 1 - isotropy_weight_H[mass_index, secondary, binE]\
                                / total_weight_H[mass_index, secondary, binE]
                        flux_per_source[mass_index, secondary, binE,s] *= J_r\
                                 /model_flux[mass_index,secondary,binE]
    return halo_flux_source, flux_per_source, model_flux

In [274]:
total_flux = np.zeros( (5, 5, Nbins_det) ); anistropy_flux = np.zeros( (5, 5, Nbins_det) )
isotropic_flux, total_flux, isotropy_weight, total_weight = Flux_ratio_calc(True)

halo_flux_source, flux_per_source, model_flux = calculate_flux(weights_H, sources,\
                                                        isotropy_weight, total_weight, flux_H, flux_L)

Calculating Flux ratio.
/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/MX_A1H.txt

/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/MX_A4H.txt

/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/MX_A14H.txt

/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/MX_A28H.txt

/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/MX_A56H.txt



# Calculating the maps

In [275]:
def LoadSmoothedMap(hp_map, radius, Nside):    
    #radius_rad = np.radians(radius_deg)
    smoothed_map = hp.sphtfunc.smoothing(hp_map, sigma = radius)
    return smoothed_map

def MapToHealpyCoord(galCoord, right_asc, declination):
    theta = np.pi/2 - declination
    phi = right_asc
    if(not galCoord): # If equatorial coordinates
        phi -= np.pi
    return phi, theta

def HealpyToMapCoord(galCoord, phi, theta ):
    declination = np.pi/2 -theta
    right_asc = phi
    if(not galCoord): # If equatorial coordinates
        right_asc += np.pi
    return right_asc, declination


import astropy.coordinates as Coord
import astropy.units as u
def Cel2Gal(ra,dec):    # arguments are Equatorial coordinates in radians
    sc_e = Coord.SkyCoord(ra*u.rad, dec*u.rad)
    lg = sc_e.galactic.l.to(u.rad).value
    lg = np.where(lg < np.pi, lg , lg - 2*np.pi)   #  [0, 2pi] -> [-pi,pi]
    bg = sc_e.galactic.b.to(u.rad).value
    return lg, bg

import sympy as smp
from scipy import special as spec
def sigma_finder(cosine):
    k = smp.symbols('k')
    #cosine = cosine if cosine>0 else 0
    cosine += 1e-9
    root = smp.nsolve( smp.coth(k)-1./k-cosine, k, cosine, prec=2)
    #print(smp.coth(root)-1./root-cosine)
    return root

def spherical_gaussian(angle, kappa):
    holder = np.exp( -(1.-np.cos(angle))*kappa )
    holder /= holder.sum()
    return holder

In [276]:
right_asc = [ source.right_ascention for source in sources]
declination = [ np.arcsin(source.sin_declination) for source in sources]
phi, theta = MapToHealpyCoord(1, np.array(right_asc), np.array(declination) )  
index = hp.ang2pix(Nside,theta, phi) 

flux_per_source_pix = np.zeros(( 5, 5, Nbins_det, len(sources),Npix ))
map_per_species_smooth = np.zeros(( 5, 5, Nbins_det, Npix ))
map_smooth = np.zeros(( 5, Nbins_det, Npix ))
final_map = np.zeros(( Nbins_det, Npix ))

theta, phi = hp.pix2ang(Nside, np.arange(Npix))
right_asc, declination =  HealpyToMapCoord(1, phi, theta )

pix_vec = np.zeros((Npix,3))
for i in range(Npix):
    pix_vec[ i , 0 ] = np.cos( np.array( right_asc[i] ) ) * np.cos( np.array( declination[i] ) )
    pix_vec[ i , 1 ] = np.sin( np.array( right_asc[i] ) ) * np.cos( np.array( declination[i] ) )
    pix_vec[ i , 2 ] = np.sin( np.array( declination[i] ) ) 

for mass_index in range(5):
    for secondary in range(mass_index+1):
        for binE in range(Nbins_det):
            for s, source in enumerate(sources):
                source_vec = np.array( ( np.cos(sources[s].right_ascention)*(1.-sources[s].sin_declination**2)**0.5,\
                    np.sin(sources[s].right_ascention)*(1.-sources[s].sin_declination**2)**0.5,\
                    sources[s].sin_declination ) )
                psi = np.arccos( (np.dot(pix_vec,source_vec)) )
                cos = abs((cosines_H[ mass_index, secondary, binE, int(source.radius)]))
                kappa = float(sigma_finder(cosine=cos) )
                map_per_species_smooth[ mass_index, secondary, binE] += spherical_gaussian(psi, kappa )*\
                            flux_per_source[ mass_index, secondary, binE, s ]

import pylab as pl

In [ ]:
"""final_map[ binE ] += fracs[mass_index]*\
    map_per_species_smooth[ mass_index, secondary, binE] #* total_weight [ mass_index, secondary, binE]\
    #/( flux_H[binE] + flux_L[binE]) + (isotropic_flux[secondary,binE]/Npix)"""

dipole = np.zeros( (5,5, 3, Nbins_det) )
dipole_module = np.zeros(Nbins_det)

for binE in range(Nbins_det):
    holder=0
    for mass_index in range(5):
        for secondary in range(mass_index+1):
            final_map[ binE ] += fracs[mass_index]*\
                map_per_species_smooth[ mass_index, secondary, binE] 
            for ra, dec, weight in zip(right_asc, declination, map_per_species_smooth[mass_index, secondary, binE]):
                dipole[mass_index,secondary,0,binE] += 3 * np.cos(ra) * np.cos(dec) * weight *\
                        whtot[mass_index, secondary,binE] / (flux_H[binE]+flux_L[binE])
                dipole[mass_index,secondary,1,binE] += 3 * np.sin(ra) * np.cos(dec) * weight *\
                        whtot[mass_index, secondary,binE] / (flux_H[binE]+flux_L[binE])
                dipole[mass_index,secondary,2,binE] += 3 * np.sin(dec) * weight *\
                        whtot[mass_index, secondary,binE] / (flux_H[binE]+flux_L[binE])
                holder += weight * fracs[mass_index]
            final_map[binE] += fracs[mass_index] * isotropic_flux[ secondary, binE]/Npix

for mass_index in range(5):
    for secondary in range(mass_index+1):
        dipole_module += fracs[mass_index] * (dipole[mass_index,secondary,0]**2+\
                        dipole[mass_index,secondary,1]**2+dipole[mass_index,secondary,2]**2)**.5

for binE in range(Nbins_det):
    print( LEd1+ dLogEii*(binE + 0.5 ), binE, np.degrees(np.arccos(dipole_module[binE]/3)), dipole_module[binE] )

for binE in range(Nbins_det,Nbins_det//1):
    hp.visufunc.mollview(final_map[binE], title=f"{18.55 +binE*0.1:5.4} < log(E/eV) < {18.65 +binE*0.1:5.4}")
    hp.visufunc.projtext(theta=theta[index[0]],phi=phi[index[0]],s="+", coord='G', color ="black", fontsize=20)
pl.show()

# Deflections in the Galactic magnetic Field

In [ ]:
path = '/home/juan/Downloads/Sibyll_Del3_NN_B50_Lc0.025_dE1_dX-1/'
dipE = np.zeros(( Nbins_det , 3 ))
vpixi = np.zeros(( 3 , Npix ))
lif = np.zeros(( 5, Nbins_det, Npix ))
bif = np.zeros(( 5, Nbins_det, Npix ))
npixf = np.zeros(( 5, Nbins_det, Npix ), dtype = int)
meanfluxE = np.zeros((Nbins_det))
lGi, bGi = Cel2Gal(np.radians(right_asc),np.radians(declination))


phi, theta = MapToHealpyCoord(1, lGi, bGi)        #healpy coord sources
index = hp.ang2pix(Nside,theta, phi)              #npix sources  (at halo)
galactic_map = np.zeros(( 5, 5, Nbins_det, Npix ))
final_galactic_map = np.zeros(( Nbins_det, Npix ))
for binE in np.arange(Nbins_det):
    if(binE == 0):
        infileZ = np.array((path+'lbo17.5.hp32',path+'lbo17.2.hp32',path+'lbo16.9.hp32',\
                                path+'lbo16.9.hp32',path+'lbo16.9.hp32'))  
    if(binE == 1):
        infileZ = np.array((path+'lbo17.8.hp32',path+'lbo17.5.hp32',path+'lbo16.9.hp32',\
                                path+'lbo16.9.hp32',path+'lbo16.9.hp32'))  
    if(binE == 2):
         infileZ = np.array((path+'lbo18.1.hp32',path+'lbo17.8.hp32',path+'lbo17.2.hp32',\
                                path+'lbo16.9.hp32',path+'lbo16.9.hp32'))  
    if(binE == 3):
        infileZ = np.array((path+'lbo18.4.hp32',path+'lbo18.1.hp32',path+'lbo17.5.hp32',\
                                path+'lbo17.2.hp32',path+'lbo16.9.hp32'))  
    if(binE == 4):
        infileZ = np.array((path+'lbo18.7.hp32',path+'lbo18.4.hp32',path+'lbo17.8.hp32',\
                                path+'lbo17.5.hp32',path+'lbo17.2.hp32'))  
    if(binE == 5):
        infileZ = np.array((path+'lbo19.hp32',path+'lbo18.7.hp32',path+'lbo18.1.hp32',\
                                path+'lbo17.8.hp32',path+'lbo17.5.hp32'))  
    if(binE == 6):
        infileZ = np.array((path+'lbo19.3.hp32',path+'lbo19.hp32',path+'lbo18.4.hp32',\
                                path+'lbo18.1.hp32',path+'lbo17.8.hp32'))  
    if(binE == 7):
        infileZ = np.array((path+'lbo19.6.hp32',path+'lbo19.3.hp32',path+'lbo18.7.hp32',\
                                path+'lbo18.4.hp32',path+'lbo18.1.hp32'))
            
    for mass_index in np.arange(5):
        for secondary in range(mass_index+1):
            mapping = np.loadtxt(infileZ[secondary])
            lif[ secondary , binE ] = np.radians(mapping[:,5])
            bif[ secondary , binE ] = np.radians(mapping[:,6])
            phif, thetaf = MapToHealpyCoord(1, lif[ secondary , binE ], bif[secondary , binE])#healpy coord sources
            npixf[secondary, binE] = hp.ang2pix(Nside,thetaf, phif)             #npix sources  (at halo)
            for pix in np.arange(Npix):
                galactic_map[mass_index, secondary, binE, pix] +=\
                    map_per_species_smooth[ mass_index, secondary, binE, npixf[secondary, binE, pix] ]
                final_galactic_map[binE, pix] += fracs[mass_index]*galactic_map[mass_index, secondary, binE, pix]+\
                     isotropic_flux[secondary, binE]/float(Npix) 

#        print(fluxepE)
    fluxtotE = np.sum(final_galactic_map, axis=1) 
    meanfluxE[binE] = fluxtotE[binE]/float(Npix)
    #final_galactic_map[binE] /= meanfluxE[binE]
    lpixi = np.radians(np.where(mapping[:,1]<180.,mapping[:,1],mapping[:,1]-360.))
    bpixi = np.radians(mapping[:,2])
    vpixi = [np.cos(bpixi)*np.cos(lpixi),np.cos(bpixi)*np.sin(lpixi),np.sin(bpixi)]
    for pix in np.arange(Npix):
        dipE[binE,0] += final_galactic_map[binE,pix]*np.cos(bpixi[pix])*np.cos(lpixi[pix])
        dipE[binE,1] += final_galactic_map[binE,pix]*np.cos(bpixi[pix])*np.sin(lpixi[pix])
        dipE[binE,2] += final_galactic_map[binE,pix]*np.sin(bpixi[pix])
    dipE[binE] = 3.*dipE[binE]#/float(Npix)

for binE in range( 0 , Nbins_det//1):
    hp.visufunc.mollview(final_galactic_map[binE],\
        title=f"{18.5 +binE*dLogEii:5.4} < log(E/eV) < {18.8 +binE*dLogEii:5.4}")
    hp.visufunc.projtext(theta=thetaf[index[0]],phi=phi[index[0]],s="+", coord='G', color ="black", fontsize=20)
pl.show()
print(dipE)